### 零、安裝套件

In [ ]:
!pip install yfinance
!pip install pandas_ta

### 壹、載入套件

In [ ]:
import yfinance as yf # 獲取股市API
import datetime as dt # 計時套件
import matplotlib.pyplot as plt # 資料視覺化套件
import pandas as pd # 資料分析套件
import numpy as np # 矩陣運算套件

### 貳、擷取股價

In [ ]:
# 輸入起始與結束時間
StartTime = dt.datetime(2010, 1, 1)
EndTime = dt.date.today()

# 輸入公司代號
CodeName = '2330.TW'

# 擷取資料
Data = yf.download(CodeName, start = StartTime, end = EndTime)

# 可視化時間範圍收盤價
plt.figure(figsize = (14, 5), dpi = 100)
plt.plot(Data['Close'], label = 'TSMC')
plt.vlines(dt.date(2016, 1, 1), 0, 270, linestyles = '--', colors = 'gray', label = 'Train/Test data cut-off')
plt.xlabel('Date')
plt.ylabel('TWD')
plt.title('Figure 1: TSMC stock price')
plt.legend()
plt.show()

### 參、技術指標

In [ ]:
def TechnicalIndicators(Data):
    # 簡單移動平均
    Data['SMA10'] = Data['Close'].rolling(window = 10).mean()
    Data['SMA40'] = Data['Close'].rolling(window = 40).mean()
    Data['SMA200'] = Data['Close'].rolling(window = 200).mean()

    # 指數移動平均
    Data['EMA12'] = Data['Close'].ewm(span = 12).mean()
    Data['EMA26'] = Data['Close'].ewm(span = 26).mean()

    # 指數平滑異同移動平均線
    Data['DIF'] = Data['EMA12'] - Data['EMA26']
    Data['MACD'] = Data['DIF'].ewm(span = 9).mean()
    Data['OSC'] = Data['DIF'] - Data['MACD']

    # 布林通道(採用SMA20跟2倍標準差)
    Data['SMA20STD'] = Data['Close'].rolling(window = 20).std()
    Data['UpperBand'] = (Data['Close'].rolling(window = 20).mean()) + (Data['SMA20STD'] * 2)
    Data['LowerBand'] = (Data['Close'].rolling(window = 20).mean()) - (Data['SMA20STD'] * 2)

    return Data

TechnicalIndicators(Data)

# 可視化技術指標(SMA、Bollinger Bands)
plt.figure(figsize = (14, 5), dpi = 100)
plt.plot(Data['Close'], label = 'Closing Price', color = 'b')
plt.plot(Data['SMA10'], label = 'MA 10', color = 'g', linestyle = '--')
plt.plot(Data['SMA40'], label = 'MA 40', color = 'r', linestyle = '--')
plt.plot(Data['SMA200'], label = 'MA 200', color = 'y', linestyle = '--')
plt.plot(Data['UpperBand'], label = 'Upper Band', color = 'c')
plt.plot(Data['LowerBand'],label = 'Lower Band', color = 'c')
Datat = Data.iloc[0:, :]
x = list(Data.index)
plt.fill_between(x, Data['LowerBand'], Data['UpperBand'], alpha = 0.3)
plt.title('Figure 2: Technical indicators(SMA、Bollinger Bands) for TSMC')
plt.xlabel('Date')
plt.ylabel('TWD')
plt.legend()
plt.show()

# 可視化技術指標(OSC)
plt.figure(figsize = (14, 5), dpi = 100)
plt.bar(x, Data['OSC'])
plt.title('Figure 3: Technical indicators(OSC) for TSMC')
plt.xlabel('Date')
plt.ylabel('TWD')
plt.legend()
plt.show()

### 肆、多支股票間的相關性

In [ ]:
# 輸入公司代號
CodeName_1 = '1216.TW'  # 統一
CodeName_2 = '1301.TW'  # 台塑
CodeName_3 = '2330.TW'  # 台積電
CodeName_4 = '2454.TW'  # 聯發科
CodeName_5 = '2603.TW'  # 長榮
CodeName_6 = '2882.TW'  # 國泰

# 輸入起始與結束時間
StartTime = dt.datetime(2015, 1, 1)
EndTime = dt.date.today()

# 擷取資料
Data_1 = yf.download(CodeName_1, start = StartTime, end = EndTime)
Data_2 = yf.download(CodeName_2, start = StartTime, end = EndTime)
Data_3 = yf.download(CodeName_3, start = StartTime, end = EndTime)
Data_4 = yf.download(CodeName_4, start = StartTime, end = EndTime)
Data_5 = yf.download(CodeName_5, start = StartTime, end = EndTime)
Data_6 = yf.download(CodeName_6, start = StartTime, end = EndTime)
Data_List = pd.DataFrame()
Data_List['1216.TW'] = Data_1['Close']
Data_List['1301.TW'] = Data_2['Close']
Data_List['2330.TW'] = Data_3['Close']
Data_List['2454.TW'] = Data_4['Close']
Data_List['2603.TW'] = Data_5['Close']
Data_List['2882.TW'] = Data_6['Close']

# 可視化相關係數矩陣
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot()
fig.colorbar(ax.matshow(Data_List.corr(), vmin = -1, vmax = 1))
ticks = np.arange(len(Data_List.corr().columns))
ax.tick_params(axis = 'x', which = 'major', labelsize = 14, rotation = 90)
ax.tick_params(axis = 'y', which = 'major', labelsize = 14, rotation = 0)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(Data_List.corr().columns)
ax.set_yticklabels(Data_List.corr().columns)
plt.title('Correlation Matrix', fontsize = 20)
plt.show()